# Image Classification CNN in Pytorch
### @2pitesh

In [15]:
# Import Libraries
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.transforms as transforms
import torchvision

In [16]:
transform = transforms.Compose([
    transforms.ToTensor(), # turns it into a tensor (norm between 0-1)
    transforms.Normalize(mean=[0.5, 0.5, 0.5], # normalize the image with a mean of 0.5 and std to 0.5 and devide by 2 to get -1 to 1
                         std=[0.5, 0.5, 0.5]),
])

In [17]:
# Retrieve the CIFAR10 dataset
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Load the dataset into a DataLoader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
# View data
image, label = train_data[0]

In [19]:
image.size() # 3 channels, 32x32 pixels (input shape)

torch.Size([3, 32, 32])

In [20]:
# Output (10 classes)
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] # labels for the 10 classes

In [22]:
# Neural Network
class NeuralNet(nn.Module):
    # Define the structure
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5) # 3 input channels, 12 output channels, 5x5 kernel
        self.pool = nn.MaxPool2d(2, 2) # 2x2 pooling
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120) # fully connected layer with 120 neurons
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # 10 output classes

    # Define the forward pass
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))

        return x

In [23]:
# Define network
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # Stochastic Gradient Descent

In [24]:
for epoch in range(30):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader): .4f}')

        # if i % 1000 == 999:
        #     print(f'Epoch {epoch}, batch {i+1}: loss {running_loss / 1000}')
        #     running_loss = 0.0

Training epoch 0...
Loss:  2.3022
Training epoch 1...
Loss:  2.2999
Training epoch 2...
Loss:  2.2841
Training epoch 3...
Loss:  2.2062
Training epoch 4...
Loss:  2.0829
Training epoch 5...
Loss:  2.0181
Training epoch 6...
Loss:  1.9718
Training epoch 7...
Loss:  1.9302
Training epoch 8...
Loss:  1.8879
Training epoch 9...
Loss:  1.8484
Training epoch 10...
Loss:  1.8131
Training epoch 11...
Loss:  1.7811
Training epoch 12...
Loss:  1.7513
Training epoch 13...
Loss:  1.7175
Training epoch 14...
Loss:  1.5898
Training epoch 15...
Loss:  1.4435
Training epoch 16...
Loss:  1.2422
Training epoch 17...
Loss:  1.1768
Training epoch 18...
Loss:  1.1381
Training epoch 19...
Loss:  1.1004
Training epoch 20...
Loss:  1.0649
Training epoch 21...
Loss:  1.0349
Training epoch 22...
Loss:  1.0056
Training epoch 23...
Loss:  0.9769
Training epoch 24...
Loss:  0.9538
Training epoch 25...
Loss:  0.9274
Training epoch 26...
Loss:  0.8181
Training epoch 27...
Loss:  0.7122
Training epoch 28...
Loss:  0.

In [25]:
# Export model parameters
torch.save(net.state_dict(), 'trained_net.pth')

In [26]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

C:\Users\valin\AppData\Local\Temp\ipykernel_49948\1690949604.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [27]:
# Test data
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 68.24%


In [ ]:
# test on example images
new_transform = transforms.Compose([
    transforms.Resize((32, 32)), # resize new images to 32x32
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=(0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    iamge = new_transform(image).unsqueeze(0)
    return image

# image_paths = []
# images = [load_image(image_path) for img in image_paths]
# net.eval()